In [17]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import requests, zipfile, io #zum abspeichern von PDFs
from tqdm import tqdm # Anzeigen des aktuellen Ladestandes
import PyPDF2
import re
from os import listdir
from os.path import isfile, join
import locale #für das deutsche Zeitformat

# 5. Alle Dataframes zusammenfügen 
- Alle einlesen und auf die Unterseiten verweisen
- eines nach dem andern anhängen
- noch schön machen und kontrollieren

In [18]:
# Einlesen:
df_v=pd.read_csv("KRFrames/df_v_Hauptseiten") #Alle Vorstösse mit Infos mit Titel und Nummer der Page mit allen Geschäften
df_dat_all=pd.read_csv("KRFrames/df_datum_ALLE")# Frame mit allen Zeitangaben(Datum)(oben)
df_np=pd.read_csv("KRFrames/Frame_Namen_Partei_endversion")# Aus File "02 Namen aus Geschäftsnamen..." Alle Namen und Parteien



# nun mal alle gleich formatieren 

In [19]:
df_v.dtypes #würde sagen das ist ok und unnamed wegnehmen

Unnamed: 0            int64
index                 int64
VorlageNR             int64
url                  object
Geschäft             object
Stand                object
Zuständigkeit        object
Art des Geschäfts    object
Status               object
dtype: object

In [20]:
df_v1=df_v[["VorlageNR", "Geschäft", "Art des Geschäfts", "Zuständigkeit", "Status", "Stand", "url"]]

In [35]:
df_v1[df_v1["Geschäft"].str.contains("Kommission")] # Ok, offenbar hat es im df_v die Kommissionsgeschäfte noch
# Die hatte ich beim anderen Dataframe rausgeputzt.

,VorlageNR,Geschäft,Art des Geschäfts,Zuständigkeit,Status,Stand,url
409,2412,Motion der Kommission zur Totalrevision der Ge...,Motion,Staatskanzlei,abgeschlossen,Erledigterklärung,/gast/geschaefte/1399
412,2408,Interpellation der ad-hoc Kommission zur Unter...,Interpellation,Finanzdirektion,abgeschlossen,Kenntnisnahme,/gast/geschaefte/1389
413,2407,Motion der ad-hoc Kommission zur Untersuchung ...,Motion,Finanzdirektion,abgeschlossen,Erledigterklärung,/gast/geschaefte/1388
523,2243,Motion der vorberatenden Kommission zum Pensio...,Motion,Finanzdirektion,abgeschlossen,Erledigterklärung,/gast/geschaefte/494
527,2238,Motion der vorberatenden Kommission zur Änderu...,Motion,Finanzdirektion,abgeschlossen,Erledigterklärung,/gast/geschaefte/489
547,2202,Motion der vorberatenden Kommission zum Integr...,Motion,Direktion für Bildung und Kultur,abgeschlossen,Erledigterklärung,/gast/geschaefte/454
592,2124,Motion der Kommission Polycom zum Kantonsratsb...,Motion,Sicherheitsdirektion,abgeschlossen,Erledigterklärung,/gast/geschaefte/376
597,2114,Motion von Thomas Aeschi betreffend Möglichkei...,Motion,Sicherheitsdirektion,abgeschlossen,Nicht-Erheblicherklärung,/gast/geschaefte/366
700,1910,Motion der erweiterten Justizprüfungskommissio...,Motion,Staatskanzlei,abgeschlossen,Erledigterklärung,/gast/geschaefte/171
977,1447,Motion der Kommission für Wasserbau und Gewäss...,Motion,NaN,abgeschlossen,Datenmigration,/gast/geschaefte/1032


In [21]:
df_dat_all.dtypes # Da sollte ich wohl datetime einstellen und die unamed wegnehmen

Unnamed: 0            int64
Unnamed: 0_x          int64
VorlageNR_x           int64
Einreichedatum       object
Link Vorstoss-PDF    object
dtype: object

In [22]:
df_dat_all["Einreichedatum"]=pd.to_datetime(df_dat_all["Einreichedatum"])

In [23]:
df_dat_all.dtypes 

Unnamed: 0                    int64
Unnamed: 0_x                  int64
VorlageNR_x                   int64
Einreichedatum       datetime64[ns]
Link Vorstoss-PDF            object
dtype: object

In [24]:
df_dat_all1=df_dat_all[["VorlageNR_x","Einreichedatum"]]
df_dat_all2=df_dat_all1.rename(columns={"VorlageNR_x": "VorlageNR"})

In [46]:
# ich mache das mit den Vorstössen mal einzeln
df_pdf_links=df_dat_all[["VorlageNR_x","Link Vorstoss-PDF"]]
df_pdf_links2=df_pdf_links.rename(columns={"VorlageNR_x": "VorlageNR"})

In [26]:
df_np.dtypes 
# Alles wegnehmen, ausser Namenganz, Partei und VorlageNR

Unnamed: 0          int64
Namenganz          object
P                  object
Unnamed: 0.1      float64
Unnamed: 0.1.1    float64
VorlageNR           int64
dtype: object

In [15]:
df_np1=df_np[["VorlageNR", "Namenganz", "P"]]

In [36]:
# Nun zusammenfügen
df_hau_dat=pd.merge(df_v1, df_dat_all2, how="inner",on= "VorlageNR")

In [43]:
df_fehlende_daten=df_hau_dat.sort_values("Einreichedatum").tail(24)

In [55]:
df_fe_da_mi_pdf=pd.merge(df_fehlende_daten, df_pdf_links2, how="inner",on= "VorlageNR")

In [58]:
df_fe_da_mi_pdf

,VorlageNR,Geschäft,Art des Geschäfts,Zuständigkeit,Status,Stand,url,Einreichedatum,Link Vorstoss-PDF
0,2718,Interpellation der Fraktion Alternative - die ...,Interpellation,Volkswirtschaftsdirektion,abgeschlossen,Kenntnisnahme,/gast/geschaefte/1707,NaT,1707
1,2718,Interpellation der Fraktion Alternative - die ...,Interpellation,Volkswirtschaftsdirektion,abgeschlossen,Kenntnisnahme,/gast/geschaefte/1707,NaT,1707
2,2718,Interpellation der Fraktion Alternative - die ...,Interpellation,Volkswirtschaftsdirektion,abgeschlossen,Kenntnisnahme,/gast/geschaefte/1707,NaT,1707
3,2718,Interpellation der Fraktion Alternative - die ...,Interpellation,Volkswirtschaftsdirektion,abgeschlossen,Kenntnisnahme,/gast/geschaefte/1707,NaT,1707
4,2526,Motion von Anna Bieri und Laura Dittli betreff...,Motion,Direktion für Bildung und Kultur,abgeschlossen,Erledigterklärung,/gast/geschaefte/1512,NaT,1512
5,2327,Kleine Anfrage der Konkordatskommission betref...,Kleine Anfrage,NaN,abgeschlossen,Datenmigration,/gast/geschaefte/597,NaT,14527_2327_1_ePolice.pdf
6,2314,Motion von Manfred Wenger betreffend Reduktion...,Motion,Sicherheitsdirektion,abgeschlossen,Nicht-Erheblicherklärung,/gast/geschaefte/563,NaT,2314-1-14497_Strassenverkehrsgeb%C3%BChren.pdf
7,2299,Kleine Anfrage von Stefan Gisler zu Aussagen d...,Kleine Anfrage,NaN,abgeschlossen,Datenmigration,/gast/geschaefte/556,NaT,14464_2299_1_Kinderbetreuung.pdf
8,2277,Interpellation von Beni Riedi betreffend Benut...,Interpellation,NaN,abgeschlossen,Datenmigration,/gast/geschaefte/531,NaT,14409_2277_1_Bostadel.pdf
9,2273,Postulat von Florian Weber und Franz Hürlimann...,Postulat,NaN,abgeschlossen,Datenmigration,/gast/geschaefte/525,NaT,14391_2273_1_Halbanschluss.pdf


In [ ]:
# Nun einen Loop bauen, um auch in diesen PDFs die Daten rauszukriegen.

In [13]:
df_all=pd.merge(df_v1, df_dat_all, how="outer",on= "VorlageNR")

In [21]:
# Offenbar hat es einzelne Vorlagenr doppelt mit mehreren Daten. ich kick die mal weg. 

df_allgut=df_all.drop_duplicates(subset='VorlageNR', keep=False) 

In [25]:
df_allgut.sort_values("Einreichedatum").tail(100)

,VorlageNR,Geschäft,Zuständigkeit,Art des Geschäfts,Status,url,Unnamed: 0,Unnamed: 0_x,Einreichedatum,Link Vorstoss-PDF
264,2626,Motion der CVP-Fraktion betreffend städtebauli...,Baudirektion,Motion,abgeschlossen,/gast/geschaefte/1609,NaN,NaN,NaN,NaN
275,2609,Motion von Philip C. Brunner und Jürg Messmer ...,Baudirektion,Motion,hängig,/gast/geschaefte/1596,NaN,NaN,NaN,NaN
277,2606,Interpellation der Fraktion Alternative – die ...,Baudirektion,Interpellation,abgeschlossen,/gast/geschaefte/1592,NaN,NaN,NaN,NaN
289,2586,"Postulat der SP-Fraktion, der SVP-Fraktion und...",Staatskanzlei,Postulat,abgeschlossen,/gast/geschaefte/1572,NaN,NaN,NaN,NaN
291,2583,"Postulat von Esther Haas, Andreas Hostettler, ...",Direktion für Bildung und Kultur,Postulat,abgeschlossen,/gast/geschaefte/1569,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1168,1101,Interpellation von Vreni Wicky betreffend Pers...,NaN,Interpellation,abgeschlossen,/gast/geschaefte/705,1154.0,1154.0,NaN,pdoc_358_1.pdf
1208,995,Motion von Alois Gössi betreffend bessere Hilf...,NaN,Motion,abgeschlossen,/gast/geschaefte/1364,1194.0,1194.0,NaN,pdoc_94_1.pdf
1221,916,"Motion von Konrad Studerus, Bruno Pezzatti und...",NaN,Motion,abgeschlossen,/gast/geschaefte/1344,1207.0,1207.0,NaN,pdoc_69_1.pdf
1232,282,Motion von Christoph Hohler betreffend Radstre...,Baudirektion,Motion,hängig,/gast/geschaefte/1712,NaN,NaN,NaN,NaN


In [45]:
df_alles.shape

(1309, 10)

In [ ]:
df_alles.to_csv("KRFrames/df_alles_ohne_Namen")

In [ ]:
df_np

In [ ]:
df_alles_ganz=pd.merge(df_alles, df_np, how="left", on="VorlageNR")
df_alles_ganz.sort_values("Einreichedatum")

In [ ]:
df_tuti=df_alles_ganz[["VorlageNR","Geschäft", "Namenganz", "P", "Einreichedatum", "Link Vorstoss-PDF"]]

In [ ]:
df_tuti #Wieso wird es nicht ergänzt. Der Fehler ist im File 02 Namen passiert. dort nachschauen.

In [ ]:

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 100)
df_tuti1=df_tuti.drop_duplicates(subset=['VorlageNR', 'Namenganz'], keep=False)
# Obwohl mir nur von 30 Geschäften das Datum fehlt, explodiert dies, wenn es auf die Namen-Tabelle kommt.
df_tuti1.sort_values("Einreichedatum").tail(188)

In [ ]:
df_tuti.to_csv("KRFrames/df_tuti_Ganzes_Frame")

# Daten aus B&A mit falschen Zahlen
Eine Lösung wäre, nochmals die Links zu den PDF auslesen und diese in zwei verschiedene Listen verpacken. 
Da die Datenmigration im Jahr 2000 war, schaue ich einfach mal die Zahlen ab 2005 an. Vo dort an, werden die meisten Geschäfte tatsächlich neu sein. 
## #Andere Variante: ich lasse die B&A einfach weg. Da gibt es eh keine verlässliche Daten. Mengenmässig.
# Nun plotten

In [ ]:
df_tuti = df_tuti.set_index('Einreichedatum')

In [ ]:
df_tuti

In [ ]:
df_tuti.dtypes